In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import seaborn.objects as so

from sklearn import linear_model    # Herramientas de modelos lineales
from sklearn.preprocessing import PolynomialFeatures    # Herramientas de polinomios
from sklearn.metrics import mean_squared_error, r2_score    # Medidas de desempeño

# Cuadrados Mínimos

### Laboratorio de Datos, IC - FCEN - UBA - 1er. Cuatrimestre 2024

Buscamos los coeficientes de un polinomio de grado $n$
$$Y = \beta_0 + \beta_1 X + \beta_2 X^2 + \beta_3 X^3 + \dots + \beta_n X^n$$

que mejor aproxime a nuestros datos en el sentido de Cuadrados Mínimos. Es decir, buscamos $\beta_1,\dots, \beta_n$ que minimicen:
$$ RSS(\beta) = \displaystyle\sum_{i=1}^n (y_i - P(x_i))^2 $$

Vamos a usar como ejemplo los datos de PBI per capita de Argentina, del dataset `gapminder`

In [ ]:
from gapminder import gapminder
datos_arg = gapminder[gapminder['country'] == 'Argentina']
datos_arg.head()

### Visualización

Teníamos la visualización de la Regresión Lineal:

In [ ]:
(
    so.Plot(data=datos_arg, x='year', y='gdpPercap')
    .add(so.Dot())
    .add(so.Line(color='r', linewidth=3), so.PolyFit(1))
)

Calculemos el $R^2$ de la Regresión Lineal:

In [ ]:
# Inicializamos el modelo de regresión
modelo = linear_model.LinearRegression()

# Realiza el ajuste
modelo.fit(datos_arg[['year']], datos_arg['gdpPercap'])

# Calculamos R²
y_pred = modelo.predict(datos_arg[['year']])
r2_score(datos_arg['gdpPercap'], y_pred)

Para cambiar el grado del polinomio que ajusta los datos, simplemente cambios el argumento de `so.PolyFit`. Por ejemplo, para un polinomio de grado $3$:

In [ ]:
(
    so.Plot(data=datos_arg, x=datos_arg['year'], y='gdpPercap')
    .add(so.Dot())
    .add(so.Line(color='r', linewidth=3), so.PolyFit(3))
)

### Cálculo de coeficientes y de predicciones

Queremos buscar el polinomio de grado 3 que mejor aproxima a los datos en sentido de cuadrados mínimos:
$$P(x) = \beta_0 + \beta_1 x + \beta_2 x^ 2 + \beta_3 x^ 3$$

Calcularemos los coeficientes con `scikit-learn`. Es parecido a lo que hacíamos con la regresión lineal, pero con un paso extra:

In [ ]:
# Indicamos que queremos un polinomio de Grado 3 sin ordenada al origen
polynomial_features= PolynomialFeatures(degree=3, include_bias=False)  

# Armamos una matriz cuya primera columna es x, la segunda es x^2 y la tercera es x^3
x_poly = polynomial_features.fit_transform(datos_arg[['year']])   
display(x_poly)

In [ ]:
# Inicializamos el modelo de regresión
modelo = linear_model.LinearRegression()

# Realiza el ajuste
modelo.fit(x_poly, datos_arg['gdpPercap'])

# Recuperamos los valores de los coeficientes (de menor potencia a la mayor)
beta = modelo.coef_

# Recuperamos la ordenada al origen
o_origen = modelo.intercept_.item()

# Imprimimos los valores:
print('beta_0: ', o_origen)
print('beta_1: ', beta[0])
print('beta_2: ', beta[1])
print('beta_3: ', beta[2])

In [ ]:
np.flip(beta)

El polinomio de grado a lo sumo 3 que mejor aproxima a los datos es:
$$P(x) = -984937719.58 + 1491410.41 x -752.77 x^ 2 + 0.12 x^ 3 $$

Corroboremos que el gráfico de este polinomio es el mismo que arma `seaborn` con `so.PolyFit`:

In [ ]:
(
    so.Plot(data=datos_arg, x='year', y='gdpPercap')
    .add(so.Dot())
    .add(so.Line(color='green', linewidth=2), y=o_origen + beta[0]*datos_arg['year'] + beta[1]*datos_arg['year']**2 + beta[2]*datos_arg['year']**3, label='sklearn')
    .add(so.Line(color='red', linestyle='--', linewidth=2), so.PolyFit(3), label='seaplot')
    .label()
)

Igual que antes, podemos estimar el PBI per capita, por ejemplo para 1990:

* haciendo las cuentas:

In [ ]:
# Estimando el PBI per capita de 1990 haciendo las cuentas con los valores de beta
o_origen + beta[0]*1990 + beta[1]*1990**2 + beta[2]*1990**3

* usando `predict`

In [ ]:
# Usando .predict()
modelo.predict([[1990, 1990**2, 1990**3]]).item()

In [ ]:
# O mas rapido, esto genera el array de potencias de 1990 desde 1 a 3 inclusive
pots = 1990 ** np.arange(1, 4)
modelo.predict([pots]).item()

* usando `numpy`

In [ ]:
# Primero tenemos que dar vuelta el vector de coeficientes
# porque numpy los toma desde el de mayor potencia al de menor
beta_flip = np.flip(beta)
print(beta)
print(beta_flip)

In [ ]:
# Segundo, agregamos al final la ordenada al origen
poly_coefs = np.concatenate((beta_flip, [o_origen]))
print(poly_coefs)

In [ ]:
# Finalmente, usamos np.polyval para evaluar el polinomio en 1990
np.polyval(poly_coefs, 1990)

También podemos calcular las medidas de desempeño del modelo:

In [ ]:
y_pred = modelo.predict(x_poly)

# Calculando el R^2
r2 = r2_score(datos_arg['gdpPercap'], y_pred)
print('R^2: ', r2)

# Calculando el ECM
ecm = mean_squared_error(datos_arg['gdpPercap'], y_pred)
print('ECM: ', ecm)